In [ ]:
from Network import *
from DriveSim import *

In [ ]:
#시뮬레이션 학습 코드
if __name__ == '__main__':
    wins = 0
    n_runs = 10000
    agent = Agent(lr=0.003, gamma=0.99, n_actions=5, batch_size=64, epsilon=1.0, input_dims=[6])
    sim = DriveSimulator()
    train_scores = []
    epsilons = []
    avg_score = 0.0
    for i in range(n_runs):
        over = False
        sim.reset(frame_rate=300)

        state = sim.get_sim_state()
        while not over:
            action = agent.choose_action(state)
            state_, stpRwd, sim_over = sim.step(action)
            over = sim_over

            agent.store_transition(state, action, stpRwd, state_, over)
            state = state_

        agent.learn()
        agent.epsilon = 1.0 - (float(i)/float(n_runs))
        #scores.append(sim.agtRwd)
        
        avg_score += sim.agtRwd
        if sim.episode_count % 100 == 0:
            print('Episode', sim.episode_count, ', average score(last 100 episodes):', avg_score/100, 'wins:', sim.win_count, 'eps:', agent.epsilon)
            train_scores.append(avg_score/100)
            epsilons.append(agent.epsilon)
            avg_score = 0.0

    agent.save_model('./models/q_20220926')

In [ ]:
plt.plot(train_scores)

In [ ]:
print(agent.q_eval.Aw.kernal)

In [ ]:
plt.plot(train_scores)


In [ ]:
#학습 결과 테스트
if __name__ == '__main__':
    wins = 0
    test_runs = 100

    sim = DriveSimulator()
    scores = []
    epsilons = []
    for i in range(test_runs):
        over = False
        sim.reset(frame_rate=30)

        state = sim.get_sim_state()
        while not over:
            action = agent.choose_action(state)
            state_, stpRwd, sim_over = sim.step(action)
            over = sim_over
            state = state_

        scores.append(sim.agtRwd)
        print('Episode', sim.episode_count, ', score:', sim.agtRwd, 'wins:', sim.win_count, 't:', sim.t)
    x = [i + 1 for i in range(test_runs)]
    plt.plot(x, scores)
    plt.show()

In [ ]:
#학습된 정책의 action 분포 측정
if __name__ == '__main__':
    test_agent = Agent(lr=0.0005, gamma=0.99, n_actions=5, batch_size=64, epsilon=0, input_dims=[6])
    test_agent.load_model('./models/q_20220926')
    wins = 0
    test_runs = 100

    sim = DriveSimulator()
    scores = []
    epsilons = []
    stats = [0,0,0,0,0]
    for i in range(test_runs):
        over = False
        sim.reset(frame_rate=30)

        state = sim.get_sim_state()
        while not over:
            action = test_agent.choose_action(state)
            stats[action] += 1
            state_, stpRwd, sim_over = sim.step(action)
            over = sim_over
            state = state_

        scores.append(sim.agtRwd)
        print('Episode', sim.episode_count, ', score:', sim.agtRwd, 'wins:', sim.win_count, 't:', sim.t)
    x = [i + 1 for i in range(test_runs)]
    plt.plot(x, scores / np.sum(scores,axis=0))
    plt.show()

    plt.bar(['stay','accel', 'decel', 'left', 'right'], stats)
    plt.show()



In [ ]:
print(stats)

#0.017 0.728 0.0026 0.119 0.134
#0.0701 0.0611 0.0065 0.1748 0.1370